In [4]:
import numpy as np
from pathlib import Path as P
import pickle
# from visualize_utils import make_vid
import cv2

    
result_path = P('/root/dj/code/CenterPoint-KITTI/output/IA-SSD-GAN-vod-aug/debug/eval/checkpoint_epoch_11')
data_path = P('/root/dj/code/CenterPoint-KITTI/data/vod_radar/training/velodyne')
gt_img_path = result_path/'gt_img'
dt_img_path = result_path/'dt_img'
gt_img_path.mkdir(exist_ok=True)
dt_img_path.mkdir(exist_ok=True)


In [2]:
data_ids = np.loadtxt(str(result_path / 'frame_ids.txt'), delimiter=',', dtype=str)[:-1]

with open(str(result_path / 'gt.pkl'), 'rb') as f:
    gt = pickle.load(f)

# load det
with open(str(result_path / 'dt.pkl'), 'rb') as f:
    dt = pickle.load(f)

In [3]:
from vod.visualization.settings import label_color_palette_2d
from matplotlib.lines import Line2D

keys = list(gt.keys())
cls_name = ['Car','Pedestrian', 'Cyclist', 'Others']
color_dict = {}
for i, v in enumerate(cls_name):
    color_dict[v] = label_color_palette_2d[v]

In [ ]:

def get_radar(fname):
    assert fname.exists()
    radar_point_cloud = np.fromfile(str(fname), dtype=np.float32).reshape(-1, 7)
    return radar_point_cloud

def get_lidar(fname):
    assert fname.exists()
    radar_point_cloud = np.fromfile(str(fname), dtype=np.float32).reshape(-1, 4)
    return radar_point_cloud

def pcd_formating(pcd):
    num_pts = pcd.shape[0]
    zeros_pad = np.zeros([num_pts, 1])
    final_pcd = np.concatenate((zeros_pad, pcd), axis=1)
    return final_pcd

In [ ]:
from visualize_point_based import drawBEV
import matplotlib.pyplot as plt
test_idx = 0

pcd_fname = data_path / (keys[test_idx] + '.bin')

radar_pcd = get_radar(pcd_fname)
radar_pcd = pcd_formating(radar_pcd)
plt.rcParams['figure.dpi'] = 150
ax = plt.gca()


drawBEV(ax, radar_pcd, None, dt[keys[test_idx]], color_dict, keys[test_idx], 'pred')
plt.xlim(-0,75)
plt.ylim(-30,30)
plt.show()


In [ ]:
ax = plt.gca()
drawBEV(ax, radar_pcd, None, gt[keys[test_idx]], color_dict, keys[test_idx], 'gt')
plt.xlim(-0,75)
plt.ylim(-30,30)
plt.show()


In [12]:
from tqdm import tqdm
def saveODImgs(frame_ids, anno, data_path, img_path, color_dict, is_radar=True, title='pred'):
    plt.rcParams['figure.dpi'] = 150
    for fid in tqdm(frame_ids):
        pcd_fname = data_path / (fid + '.bin')
        vis_pcd = get_radar(pcd_fname) if is_radar else get_lidar(pcd_fname)
        vis_pcd = pcd_formating(vis_pcd)
        ax = plt.gca()
        drawBEV(ax, vis_pcd, None, anno[fid], color_dict, fid, title)
        plt.xlim(-0,75)
        plt.ylim(-30,30)
        img_fname = img_path / (fid + '.png')
        plt.savefig(str(img_fname))
        plt.cla()

In [13]:
saveODImgs(data_ids, gt, data_path, gt_img_path, \
    color_dict, is_radar=True, title='gt')

NameError: name 'data_ids' is not defined

In [ ]:
saveODImgs(data_ids, dt, data_path, dt_img_path, \
    color_dict, is_radar=True, title='pred CFAR')

In [8]:
from glob import glob
gt_imgs = sorted(glob(str(gt_img_path/'*.png')))
dt_imgs = sorted(glob(str(dt_img_path/'*.png')))

In [11]:


def make_vid(imgs, vid_fname, fps=15):
    out = None
    for fname in imgs:
        i = cv2.imread(fname)
        if out is None:
            h, w, _ = i.shape
            size = (w, h)
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(str(vid_fname), fourcc, fps, size)
            
        out.write(i)
    out.release()
make_vid(gt_imgs, result_path/'gt.mp4', fps=10)
make_vid(dt_imgs, result_path/'dt.mp4', fps=10)